In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Based in parts on...

[Ubiquant Market Prediction with Ridge Regression](https://www.kaggle.com/lonnieqin/ubiquant-market-prediction-with-ridge-regression)

[Ubiquant LGBM Baseline](https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline)

[Principal Component Regression](https://www.kaggle.com/phamvanvung/principal-component-regression)

Various StackOverflow posts

In [ ]:
import numpy as np

VIF = False
PCR = False
VIF_THRESHOLD = 25
SAMPLE_SIZE = 3000000
REGR_ONLY = True
DATA_TYPE = np.float32

In [ ]:
import os
import gc
import joblib
import random
import pandas as pd
from pathlib import Path
from argparse import Namespace
from collections import defaultdict
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 64)

def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

def reduce_memory_usage(df, features):
    start_mem = df.memory_usage().sum() / 1024**2
    for feature in features:
        item = df[feature].astype(DATA_TYPE)
        df[feature] = item
        del item
        gc.collect()
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

For PCR different sample sizes seem possible for different VIF_THRESHOLD settings. The following seems true. Your result might vary?

In [ ]:
#if VIF and PCR:
#    if VIF_THRESHOLD > 4 and SAMPLE_SIZE > 2800000:
#        SAMPLE_SIZE = 2800000
#elif VIF:
#    if VIF_THRESHOLD > 8 and SAMPLE_SIZE > 2800000:
#        SAMPLE_SIZE = 2800000
    
#print("For VIF " + str(VIF) + " and PCR " + str(PCR) + " with VIF_THRESHOLD " + str(VIF_THRESHOLD) + " SAMPLE_SIZE set to " + str(SAMPLE_SIZE))   
if PCR:
    SAMPLE_SIZE = 2250000

In [ ]:
# Rows and Columns in train dataset: (3141410, 304)
args = Namespace(
    seed=21,
    folds=5,
    workers=4,
    samples=SAMPLE_SIZE,
    data_path=Path("../input/ubiquant-parquet/"),
)
seed_everything(args.seed)

In [ ]:
%%time
train = pd.read_parquet(args.data_path.joinpath("train_low_mem.parquet"))
assert train.isnull().any().sum() == 0, "null exists."

In [ ]:
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
feature_columns = ['investment_id', 'time_id'] + features
reduce_memory_usage(train, features + ["target"])
#train = reduce_mem_usage(train)
#gc.collect()

In [ ]:
import tensorflow as tf
from sklearn import metrics

def symmetric_mean_absolute_percentage_error(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
def evaluate(model, x_val, y_val):
    y_pred = model.predict(x_val)
    r2 = metrics.r2_score(y_val, y_pred)
    mse = metrics.mean_squared_error(y_val, y_pred)
    mae = metrics.mean_absolute_error(y_val, y_pred)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_val, y_pred).numpy()
    rmse = np.sqrt(mse)
    smape = symmetric_mean_absolute_percentage_error(y_val, y_pred)
    print("R2 Score:", r2)
    print("MSE:", mse)
    print("MAE:", mae)
    print("MAPE", mape)
    print("RMSE:", rmse)
    print("SMAPE:", smape)
    return {"r2": r2, "mse": mse, "mae": mae, "mape": mape, "rmse": rmse, "smape": smape}

***VIF***

In [ ]:
if VIF:
    corr_matrix = train.iloc[1:1800000,4:].corr()
    vif = pd.Series(np.linalg.inv(corr_matrix.values).diagonal(),index=train.columns[4:])
    # colinear test
    col = vif[vif.values >= VIF_THRESHOLD]
    print(col.iloc[np.lexsort([col.index,col.values])])
    del corr_matrix
    del vif
    gc.collect()
    train.drop(columns=col.index,inplace=True)
    print(train.shape)

In [ ]:
if args.samples is not None:
    #train = train.sample(args.samples, random_state=args.seed).reset_index(drop=True)
    train = train[-args.samples:].reset_index(drop=True)
    gc.collect()
train.shape

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
y = train["target"]
X = train.drop(["row_id","time_id","investment_id","target"], axis = 1).astype(DATA_TYPE)
del train
gc.collect()
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

#training = df.copy()

#print("training", training.shape)
del y_test
del X
del y
gc.collect()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import scale 
#pca = PCA()
ipca = IncrementalPCA(batch_size=1000) 

In [ ]:
if PCR:
    X_regr = ipca.fit_transform(scale(X_train))
    del X_train
    gc.collect()
else:
    print("X_train",np.count_nonzero(np.isnan(X_train)))
    X_regr = scale(X_train)
    print("X_regr",np.count_nonzero(np.isnan(X_regr)))
X_regr.shape

In [ ]:
if PCR:
    np.cumsum(np.round(ipca.explained_variance_ratio_, decimals = 4)*100)[0:5]

In [ ]:
%%time
from sklearn.ensemble import VotingRegressor 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import Ridge, LinearRegression, ElasticNet, Lasso
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
kfold = StratifiedKFold(5, shuffle=True, random_state=42)
from sklearn.pipeline import Pipeline
if not REGR_ONLY:
    models = []
    use_k_fold = False
    if use_k_fold:
        for (train_indices, valid_indices) in kfold.split(X, X["investment_id"]):
            X_train, X_val = X.iloc[train_indices], X.iloc[valid_indices]
            y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
            # MinMaxScaler
            # RobustScaler
            model = Pipeline([('scaler', RobustScaler()), ('ridge', Ridge())])
            model.fit(X_train, y_train)
            evaluate(model, X_val, y_val)
            models.append(model)
            del X_train
            del X_val
            del y_train
            del y_val
            gc.collect()
    else:
        #model = Ridge()
        model = StackingRegressor([
            ("ridge", Ridge()), 
            ("lr", LinearRegression()),
            ("elastic", ElasticNet()),
            ("lasso", Lasso())
        ])
        #model = Pipeline([('scaler', StandardScaler()), ('vr', vr)])
        #model.fit(X, y)
        print(X_regr.shape)
        model.fit(X_regr, y_train)
        #evaluate(model, X, y)
        evaluate(model, X_regr, y_train)
        y_pred = model.predict(X_regr)
        print(pearsonr(y_train, y_pred)[0])
        models.append(model)

In [ ]:
if REGR_ONLY:
    lm = LinearRegression()
    regr_model = lm.fit(X_regr, y_train)

    print(regr_model.intercept_)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
if REGR_ONLY:
    print(regr_model.coef_)
    y_pred = regr_model.predict(X_regr)
    y_pred[0:5]
    print(np.sqrt(mean_squared_error(y_train, y_pred)))
    print(pearsonr(y_train, y_pred)[0])

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

#scaler = joblib.load('scaler.joblib')
#models = [joblib.load(f'lgbm_seed{args.seed}_{fold}.pkl') for fold in range(args.folds)]
scaler = StandardScaler()
if REGR_ONLY:
    model = regr_model
for (test_df, sample_prediction_df) in iter_test:
    if VIF:
        test_df.drop(columns=col.index,inplace=True)
    num_features = list(test_df.filter(like="f_").columns)
    test_df[num_features] = scale(test_df[num_features]) 
    #final_pred = [models[fold].predict(test_df[num_features]) for fold in range(args.folds)]
    sample_prediction_df['target'] = model.predict(test_df[num_features])
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)